In [1]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
from math import cos, sin, radians
from collections import deque
import itertools

In [2]:
# Params
N_FRAMES = 2
GAMMA = 0.95
BATCH_SIZE = 64
LEARNING_RATE = 0.00025
MAX_MEMORY = 200_000
MIN_REPLAY_SIZE = 100_000 * N_FRAMES
EPSILON_START = 1.0
EPSILON_END = 0.02
EPSILON_DECAY = 80000 * N_FRAMES
TARGET_UPDATE_FREQ = 10000 * N_FRAMES

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
# Game Logic
class Game:
    def __init__(self, play, train):
        self.play = play
        self.train = train
        self.screen = None
        self.player1 = Paddle(self.screen, 5)
        self.player2 = Paddle(self.screen, 625)
        self.ball = Ball(self.screen)

    def update(self):
        self.player1.update()
        self.player2.update()
        reward, done, score = self.ball.update(self.player1, self.player2)
        return reward, done, score

    def getState(self):
        state = [
            round(self.player1.y / (480 - self.player1.height), 2),
            round(self.player2.y / (480 - self.player2.height), 2),
            round(self.ball.y / 480, 2),
            round(self.ball.x / 640, 2),
            round(self.ball.angle / 255, 2),
        ]
        return np.array(state, dtype=np.float)

    def run(self):
        reward, done, score = self.update()
        return reward, done, score

class Ball:
    def __init__(self, screen):
        self.screen = screen
        self.frame = 0
        self.x = 320
        self.y = 240
        self.angle = random.randint(-45, 45) + 180 * random.randint(0, 1)
        self.speed = 8
        self.radius = 6
        self.scoreDiff = 0

    def update(self, player1, player2):
        reward = 0
        done = False
        self.frame += 1
        
        # Check if ball hits the top or bottom
        if self.y + self.radius > 480 or self.y - self.radius < 0:
            self.angle = -self.angle

        # left collide
        if self.x - self.radius >= player1.x and self.x - self.radius <= player1.x + player1.width:
            if self.y - player1.y >= -self.radius:
                if self.y - player1.y <= 1/8 * (player1.height + self.radius):
                    self.angle = -45

                elif self.y - player1.y <= 2/8 * (player1.height + self.radius):
                    self.angle = -30

                elif self.y - player1.y <= 3/8 * (player1.height + self.radius):
                    self.angle = -15

                elif self.y - player1.y <= 4/8 * (player1.height + self.radius):
                    self.angle = -10

                elif self.y - player1.y <= 5/8 * (player1.height + self.radius):
                    self.angle = 10

                elif self.y - player1.y <= 6/8 * (player1.height + self.radius):
                    self.angle = 15

                elif self.y - player1.y <= 7/8 * (player1.height + self.radius):
                    self.angle = 30

                elif self.y - player1.y <= 8/8 * (player1.height + self.radius):
                    self.angle = 45
                reward = 2

        # right collide
        elif self.x + self.radius >= player2.x and self.x + self.radius <= player2.x + player2.width:
            if self.y - player2.y >= -self.radius:
                if self.y - player2.y <= 1/8 * (player2.height + self.radius):
                    self.angle = -135

                elif self.y - player2.y <= 2/8 * (player2.height + self.radius):
                    self.angle = -150

                elif self.y - player2.y <= 3/8 * (player2.height + self.radius):
                    self.angle = -165

                elif self.y - player2.y <= 4/8 * (player2.height + self.radius):
                    self.angle = 170

                elif self.y - player2.y <= 5/8 * (player2.height + self.radius):
                    self.angle = 190

                elif self.y - player2.y <= 6/8 * (player2.height + self.radius):
                    self.angle = 165

                elif self.y - player2.y <= 7/8 * (player2.height + self.radius):
                    self.angle = 150

                elif self.y - player2.y <= 8/8 * (player2.height + self.radius):
                    self.angle = 135

        self.y += self.speed*sin(radians(self.angle))
        self.x += self.speed*cos(radians(self.angle))

        # Check if the Ball went right
        if self.x - self.radius >= 670:
            player1.score += 1
            reward = 10
            self.scoreDiff += 1
            self.x = player2.x - player2.width * 2 - self.radius
            self.y = 240
            self.angle = 180
            self.frame = 0

        temp = self.scoreDiff
        
        # Check if the Ball went left
        if self.x + self.radius <= -30 or self.frame > 1000:
            player2.score += 1
            reward = -10
            if player2.score % 5 == 0:
                done = True
                self.scoreDiff = 0
            self.x = player1.x + player1.width * 2 + self.radius
            self.y = 240
            self.angle = 0
            self.frame = 0

        return reward, done, temp

class Paddle:
    def __init__(self, screen, x):
        self.screen = screen
        self.x = x
        self.speed = 4
        self.width = 10
        self.height = 80
        self.y = 240 - self.height / 2
        self.score = 0
        self.mode = 0

    def update(self):
        self.y += self.mode * self.speed
        self.y = max(0, min(self.y, 480 - self.height))


In [4]:
# DQN
class Linear_QNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1024)
        self.linear4 = nn.Linear(1024, 256)
        self.linear5 = nn.Linear(256, output_size)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = self.linear5(x)
        return x

    def act(self, state):
        with torch.no_grad():
            state_t = torch.as_tensor(state, dtype=torch.float32, device=DEVICE)
            q_values = self(state_t.unsqueeze(0))
            max_q_index = torch.argmax(q_values, dim=1)[0]
            action = max_q_index.detach().item()
        return action

    def save(self, file_name='model.pth'):
        model_folder_path = './model'
        if not os.path.exists(model_folder_path):
            os.makedirs(model_folder_path)
        file_name = os.path.join(model_folder_path, file_name)
        torch.save(self.state_dict(), file_name)


In [5]:
def main(play, train):
    game = Game(play, train)
    online_net = Linear_QNet(5, 2).to(DEVICE)
    run = True
    target_net = Linear_QNet(5, 2).to(DEVICE)
    target_net.load_state_dict(online_net.state_dict())
    memory = deque(maxlen=MAX_MEMORY)
    optimizer = optim.Adam(online_net.parameters(), lr=LEARNING_RATE)
    record = 0
    for step in itertools.count():
        if not run:
            break
        if game.ball.y < game.player2.y + game.player2.height/2:
            game.player2.mode = -1
        elif game.ball.y > game.player2.y - game.player2.height/2:
            game.player2.mode = 1

        # Load with random actions at first
        if step % N_FRAMES == 0:
            if step < MIN_REPLAY_SIZE:
                state_old = game.getState()
                action = random.randint(0, 1)
                if action == 0:
                    game.player1.mode = -1
                else:
                    game.player1.mode = 1
                reward, done, score = game.run()
                state_new = game.getState()
                memory.append((state_old, action, reward, done, state_new))
                state_old = state_new

            else:
                state_old = game.getState()
                epsilon = np.interp(
                    step-MIN_REPLAY_SIZE, [0, EPSILON_DECAY], [EPSILON_START, EPSILON_END])
                if random.random() <= epsilon:
                    action = random.randint(0, 1)
                else:
                    action = online_net.act(state_old)
                if action == 0:
                    game.player1.mode = -1
                else:
                    game.player1.mode = 1

                reward, done, score = game.run()
                state_new = game.getState()
                memory.append((state_old, action, reward, done, state_new))
                state_old = state_new

                transitions = random.sample(memory, BATCH_SIZE)

                obses = np.asarray([t[0] for t in transitions])
                actions = np.asarray([t[1] for t in transitions])
                rews = np.asarray([t[2] for t in transitions])
                dones = np.asarray([t[3] for t in transitions])
                new_obses = np.asarray([t[4] for t in transitions])

                obses_t = torch.as_tensor(obses, dtype=torch.float32, device=DEVICE)
                actions_t = torch.as_tensor(actions, dtype=torch.int64, device=DEVICE).unsqueeze(-1)
                rews_t = torch.as_tensor(rews, dtype=torch.float32, device=DEVICE).unsqueeze(-1)
                dones_t = torch.as_tensor(dones, dtype=torch.float32, device=DEVICE).unsqueeze(-1)
                new_obses_t = torch.as_tensor(new_obses, dtype=torch.float32, device=DEVICE)

                # Compute Targets
                # targets = r + gamma * target q vals * (1 - dones)
                target_q_values = target_net(new_obses_t)
                max_target_q_values = target_q_values.max(
                    dim=1, keepdim=True)[0]
                targets = rews_t + GAMMA * \
                    (1 - dones_t) * max_target_q_values

                # Compute Loss
                q_values = online_net(obses_t)
                action_q_values = torch.gather(
                    input=q_values, dim=1, index=actions_t)
                loss = nn.functional.smooth_l1_loss(
                    action_q_values, targets)

                # Gradient Descent
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Update Target Net
                if max(1, step-MIN_REPLAY_SIZE) % TARGET_UPDATE_FREQ == 0:
                    print(step, epsilon)
                    target_net.load_state_dict(online_net.state_dict())

                if done:
                    if score > record:
                        record = score
                        print('Record:', record, 'Step:', step)
                        online_net.save()

        else:
            reward, done, score = game.run()

In [6]:
main(False, True)

282000 0.9755
284000 0.951
286000 0.9265
288000 0.902
290000 0.8775
292000 0.853
294000 0.8285
296000 0.804
298000 0.7795
300000 0.755
Record: 1 Step: 300612
302000 0.7304999999999999
304000 0.706
306000 0.6815
308000 0.657
310000 0.6325000000000001
312000 0.608
Record: 2 Step: 312910
314000 0.5835
316000 0.5589999999999999
318000 0.5345
320000 0.51
322000 0.48550000000000004
324000 0.46099999999999997
326000 0.4365
328000 0.41200000000000003
330000 0.38750000000000007
332000 0.363
334000 0.3385
336000 0.31400000000000006
338000 0.2895
340000 0.265
Record: 3 Step: 341096
342000 0.24050000000000005
344000 0.21599999999999997
346000 0.1915
348000 0.16700000000000004
350000 0.14250000000000007
352000 0.118
354000 0.09350000000000003
356000 0.06900000000000006
358000 0.044499999999999984
360000 0.02
362000 0.02
364000 0.02
366000 0.02
368000 0.02
370000 0.02
372000 0.02
374000 0.02
376000 0.02
378000 0.02
380000 0.02
382000 0.02
384000 0.02
386000 0.02
388000 0.02
390000 0.02
392000 0.02
3

KeyboardInterrupt: ignored